## Setup the environment by loading the packages

In [5]:

# Core
import pandas as pd
import numpy as np
from plotly import express as px
from utils import *

# compress the warnings
import warnings
warnings.filterwarnings('ignore')

## Load the data

In [6]:
df = pd.read_csv("data/ga_bigquery_data.csv")
df.head()

,fullVisitorId,date,channelGrouping,traffic_source,total_transactions,total_transaction_revenue
0,4702386946621457676,20170103,Organic Search,(direct),NaN,NaN
1,2087993472864421231,20170103,Organic Search,(direct),NaN,NaN
2,0577469839995590230,20170103,Organic Search,(direct),NaN,NaN
3,7618446014168949772,20170103,Paid Search,(direct),NaN,NaN
4,4837057017588527755,20170103,Paid Search,(direct),NaN,NaN


In [7]:
df.columns

Index(['fullVisitorId', 'date', 'channelGrouping', 'traffic_source',
       'total_transactions', 'total_transaction_revenue'],
      dtype='object')

In [69]:
activity_df = df[['fullVisitorId', 'date', 'channelGrouping', 'total_transactions']].copy()
# rename the columns
activity_df.columns = ['id', 'date', 'state', 'total_transactions']
# 'date' column is in string format, 'yyyymmdd' , need to convert it to datetime
activity_df['date'] = pd.to_datetime(activity_df['date'], format='%Y%m%d')

# sort the dataframe by id and date
activity_df = activity_df.sort_values(['id', 'date'])
activity_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 847224 entries, 661756 to 206123
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  847224 non-null  object        
 1   date                847224 non-null  datetime64[ns]
 2   state               847224 non-null  object        
 3   total_transactions  11458 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 32.3+ MB


In [70]:
activity_df.head()

,id,date,state,total_transactions
661756,4823595352351,2016-11-01,Organic Search,NaN
639230,5103959234087,2016-08-21,Organic Search,NaN
622488,33471059618621,2016-11-27,Social,NaN
833474,35794135966385,2017-01-20,Direct,NaN
811487,39460501403861,2017-03-27,Social,NaN


In [72]:

# Calculate 'first_transaction_date' for 'total_transactions' > 0
first_transaction_dates = activity_df.loc[activity_df['total_transactions'] > 0].groupby('id')['date'].min()

# Assign these dates to all rows with the same 'id'
activity_df['first_transaction_date'] = activity_df['id'].map(first_transaction_dates)

# activity_df['first_transaction_date'] = activity_df['first_transaction_date'].where(activity_df['total_transactions']>0, pd.NaT)
activity_df['first_transaction_date'] = pd.to_datetime(activity_df['first_transaction_date'])
activity_df['is_active'] = activity_df['date'] <= activity_df['first_transaction_date']
# calculate the cumulative transactions
activity_df['cumulative_transactions'] = activity_df.groupby('id')['total_transactions'].cumsum()

# delete all the rows where date is greater than first_transaction_date if first_transaction_date is not NaT
activity_df = activity_df.loc[~((activity_df['date'] > activity_df['first_transaction_date']) & activity_df['first_transaction_date'].notna())]
# activity_df = activity_df[activity_df['date'] <= activity_df['first_transaction_date']]
# if cumulative_transactions > 0, is_converted = 1; else 0
activity_df['is_converted'] = activity_df['cumulative_transactions'] > 0
activity_df['is_converted'] = activity_df['is_converted'].astype(int)
activity_df.head()

,id,date,state,total_transactions,first_transaction_date,is_active,cumulative_transactions,is_converted
661756,4823595352351,2016-11-01,Organic Search,NaN,NaT,False,NaN,0
639230,5103959234087,2016-08-21,Organic Search,NaN,NaT,False,NaN,0
622488,33471059618621,2016-11-27,Social,NaN,NaT,False,NaN,0
833474,35794135966385,2017-01-20,Direct,NaN,NaT,False,NaN,0
811487,39460501403861,2017-03-27,Social,NaN,NaT,False,NaN,0


In [75]:
activity_df.loc[activity_df['id'] == 14262055593378383]

,id,date,state,total_transactions,first_transaction_date,is_active,cumulative_transactions,is_converted
575285,14262055593378383,2017-06-20,Referral,NaN,2017-06-27,True,NaN,0
599842,14262055593378383,2017-06-27,Referral,1.0,2017-06-27,True,1.0,1


In [73]:
activity_df.loc[activity_df['cumulative_transactions'] > 1].head()

,id,date,state,total_transactions,first_transaction_date,is_active,cumulative_transactions,is_converted
396213,24932550342595467,2017-07-22,Paid Search,2.0,2017-07-22,True,2.0,1
406475,43584487743669327,2017-06-19,Referral,2.0,2017-06-19,True,2.0,1
800129,56746998029721050,2016-10-31,Organic Search,2.0,2016-10-31,True,2.0,1
726001,62231184035350130,2016-08-17,Direct,2.0,2016-08-17,True,2.0,1
650770,68437640696494161,2016-12-14,Organic Search,5.0,2016-12-14,True,5.0,1


## Transform the data to get the customer Journey dataframe

In [76]:
activity_df = activity_df[['id', 'date', 'state', 'is_converted']].copy()

In [77]:

class CustomerJourneyTransformer:
    def __init__(self, activity_df):
        self.activity_df = activity_df
    
    def transform(self):
        # Sort the dataframe by 'id' and 'date'
        self.activity_df.sort_values(by=['id', 'date'], inplace=True)
        
        # Create a new dataframe with 'next_state_date', 'next_state', and 'journey_order_number' columns
        journey_df = self.activity_df.assign(
            next_state_date=self.activity_df.groupby('id')['date'].shift(-1),
            next_state=self.activity_df.groupby('id')['state'].shift(-1),
            next_converted = self.activity_df.groupby('id')['is_converted'].shift(-1),
            journey_order_number=self.activity_df.groupby('id').cumcount() + 1 
        
        )

        journey_df['next_journey_order_number'] = journey_df.groupby('id')['journey_order_number'].shift(-1)
                
        # delete rows where is_converted = 1
        # journey_df = journey_df[journey_df['is_converted'] == 0]
        
        return journey_df


In [78]:

# Create an instance of CustomerJourneyTransformer
transformer = CustomerJourneyTransformer(activity_df)

# Transform the activity dataframe
journey_df = transformer.transform()

journey_df.head()

,id,date,state,is_converted,next_state_date,next_state,next_converted,journey_order_number,next_journey_order_number
661756,4823595352351,2016-11-01,Organic Search,0,NaT,NaN,NaN,1,NaN
639230,5103959234087,2016-08-21,Organic Search,0,NaT,NaN,NaN,1,NaN
622488,33471059618621,2016-11-27,Social,0,NaT,NaN,NaN,1,NaN
833474,35794135966385,2017-01-20,Direct,0,NaT,NaN,NaN,1,NaN
811487,39460501403861,2017-03-27,Social,0,NaT,NaN,NaN,1,NaN


In [80]:
# create a new column 'final_status' : max(journey_df['is_converted']) by id 
final_status = journey_df.groupby('id')['is_converted'].max()
journey_df['final_status'] = journey_df['id'].map(final_status)
journey_df['final_status'] = journey_df['final_status'].astype(int)
#  for final_status = 0: next_state = 'not_converted'
journey_df['next_state'] = journey_df['next_state'].where(journey_df['final_status'] == 1, 'not_converted')
# for final_status = 1: if state is not null and next_state is null: next_state = 'converted'
journey_df.loc[journey_df['final_status'] == 1, 'next_state'] = journey_df.loc[journey_df['final_status'] == 1, 'next_state'].where(journey_df['next_state'].notna(), 'converted')
journey_df.head()

,id,date,state,is_converted,next_state_date,next_state,next_converted,journey_order_number,next_journey_order_number,final_status
661756,4823595352351,2016-11-01,Organic Search,0,NaT,not_converted,NaN,1,NaN,0
639230,5103959234087,2016-08-21,Organic Search,0,NaT,not_converted,NaN,1,NaN,0
622488,33471059618621,2016-11-27,Social,0,NaT,not_converted,NaN,1,NaN,0
833474,35794135966385,2017-01-20,Direct,0,NaT,not_converted,NaN,1,NaN,0
811487,39460501403861,2017-03-27,Social,0,NaT,not_converted,NaN,1,NaN,0


In [67]:
journey_df.loc[journey_df.final_status == 0].head()

,id,date,state,is_converted,next_state_date,next_state,next_converted,journey_order_number,next_journey_order_number,final_status


In [81]:
journey_df.loc[journey_df['id'] == 14262055593378383]

,id,date,state,is_converted,next_state_date,next_state,next_converted,journey_order_number,next_journey_order_number,final_status
575285,14262055593378383,2017-06-20,Referral,0,2017-06-27,Referral,1.0,1,2.0,1
599842,14262055593378383,2017-06-27,Referral,1,NaT,converted,NaN,2,NaN,1


In [64]:
journey_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15707 entries, 675011 to 170650
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         15707 non-null  object        
 1   date                       15707 non-null  datetime64[ns]
 2   state                      15707 non-null  object        
 3   is_converted               15707 non-null  int64         
 4   next_state_date            5147 non-null   datetime64[ns]
 5   next_state                 5147 non-null   object        
 6   next_converted             5147 non-null   float64       
 7   journey_order_number       15707 non-null  int64         
 8   next_journey_order_number  5147 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(2), object(3)
memory usage: 1.2+ MB


In [82]:
# show all the rows where id is '166465265517759' 
journey_df.loc[journey_df['id'] == 3450834640354121]



,id,date,state,is_converted,next_state_date,next_state,next_converted,journey_order_number,next_journey_order_number,final_status
441718,3450834640354121,2016-08-16,Organic Search,0,2016-09-05,Organic Search,0.0,1,2.0,1
530027,3450834640354121,2016-09-05,Organic Search,0,2016-09-08,Organic Search,0.0,2,3.0,1
644333,3450834640354121,2016-09-08,Organic Search,0,2016-09-09,Organic Search,0.0,3,4.0,1
673380,3450834640354121,2016-09-09,Organic Search,0,2016-09-16,Organic Search,0.0,4,5.0,1
476031,3450834640354121,2016-09-16,Organic Search,0,2016-10-18,Organic Search,1.0,5,6.0,1
556558,3450834640354121,2016-10-18,Organic Search,1,NaT,converted,NaN,6,NaN,1


In [83]:
journey_df.loc[journey_df['journey_order_number'] > 1].head()

,id,date,state,is_converted,next_state_date,next_state,next_converted,journey_order_number,next_journey_order_number,final_status
540561,166465265517759,2016-09-15,Organic Search,0,NaT,not_converted,NaN,2,NaN,0
500024,170187170673177,2017-06-10,Referral,0,2017-06-26,not_converted,0.0,2,3.0,0
449567,170187170673177,2017-06-26,Referral,0,2017-07-25,not_converted,0.0,3,4.0,0
628376,170187170673177,2017-07-25,Referral,0,NaT,not_converted,NaN,4,NaN,0
705167,307556418998859,2016-11-04,Organic Search,0,NaT,not_converted,NaN,2,NaN,0


In [84]:
journey_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 841491 entries, 661756 to 206123
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   id                         841491 non-null  object        
 1   date                       841491 non-null  datetime64[ns]
 2   state                      841491 non-null  object        
 3   is_converted               841491 non-null  int64         
 4   next_state_date            82697 non-null   datetime64[ns]
 5   next_state                 841491 non-null  object        
 6   next_converted             82697 non-null   float64       
 7   journey_order_number       841491 non-null  int64         
 8   next_journey_order_number  82697 non-null   float64       
 9   final_status               841491 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(3), object(3)
memory usage: 102.9+ MB


In [85]:
journey_df.tail()

,id,date,state,is_converted,next_state_date,next_state,next_converted,journey_order_number,next_journey_order_number,final_status
209857,9999761280717362235,2016-09-28,Paid Search,0,2016-12-12,not_converted,0.0,1,2.0,0
273336,9999761280717362235,2016-12-12,Paid Search,0,NaT,not_converted,NaN,2,NaN,0
15721,9999824060591971928,2017-06-14,Organic Search,0,NaT,not_converted,NaN,1,NaN,0
235945,999997225970956660,2016-08-19,Organic Search,0,NaT,not_converted,NaN,1,NaN,0
206123,9999978264901065827,2017-01-24,Organic Search,0,NaT,not_converted,NaN,1,NaN,0


## create the nodes (each step counts) and flows (step to step counts)

In [89]:
# https://github.com/riley-x/SankeyFlow

## create the nodes (each step counts) and flows (step to step counts)
journey_df['source'] = journey_df['journey_order_number'].astype(str) + '-' + journey_df['state']
journey_df['target'] = journey_df['next_journey_order_number'].astype(str) + '-' + journey_df['next_state']



In [90]:
# create the nodes: counts by source
nodes_df = journey_df.groupby(['journey_order_number', 'source'])['id'].count().reset_index()
nodes_df.head()


,journey_order_number,source,id
0,1,1-(Other),56
1,1,1-Affiliates,13565
2,1,1-Direct,115594
3,1,1-Display,3101
4,1,1-Organic Search,322641


In [96]:
max(nodes_df['journey_order_number'])

66

In [94]:
# nodes is a nested list of length nlevels, ordered from left to right. For each level, there is a list of nodes ordered from top to bottom. Each node is a (name, value) pair.
nodes = []
for level in nodes_df['journey_order_number'].unique():
    df_level = nodes_df.loc[nodes_df['journey_order_number'] == level]
    node = [(source, count) for source, count in zip(df_level['source'], df_level['id'])]
    nodes.append(node)

In [95]:
nodes

[[('1-(Other)', 56),
  ('1-Affiliates', 13565),
  ('1-Direct', 115594),
  ('1-Display', 3101),
  ('1-Organic Search', 322641),
  ('1-Paid Search', 18033),
  ('1-Referral', 70419),
  ('1-Social', 215385)],
 [('2-(Other)', 30),
  ('2-Affiliates', 1051),
  ('2-Direct', 8899),
  ('2-Display', 1340),
  ('2-Organic Search', 21179),
  ('2-Paid Search', 2708),
  ('2-Referral', 13625),
  ('2-Social', 5165)],
 [('3-(Other)', 9),
  ('3-Affiliates', 230),
  ('3-Direct', 2201),
  ('3-Display', 404),
  ('3-Organic Search', 5219),
  ('3-Paid Search', 658),
  ('3-Referral', 4034),
  ('3-Social', 953)],
 [('4-(Other)', 4),
  ('4-Affiliates', 82),
  ('4-Direct', 946),
  ('4-Display', 150),
  ('4-Organic Search', 1964),
  ('4-Paid Search', 300),
  ('4-Referral', 1615),
  ('4-Social', 311)],
 [('5-(Other)', 1),
  ('5-Affiliates', 36),
  ('5-Direct', 516),
  ('5-Display', 82),
  ('5-Organic Search', 964),
  ('5-Paid Search', 145),
  ('5-Referral', 785),
  ('5-Social', 151)],
 [('6-Affiliates', 21),
  ('6-D

In [100]:
# flows: counts by source and target
flows_df = journey_df.groupby(['source', 'target'])['id'].count().reset_index()
# target column: change 'nan-converted' to 'converted', change 'nan-not_converted' to 'not_converted'
flows_df['target'] = flows_df['target'].str.replace('nan-converted', 'converted')
flows_df['target'] = flows_df['target'].str.replace('nan-not_converted', 'not_converted')
flows_df.head()


,source,target,id
0,1-(Other),2.0-Organic Search,1
1,1-(Other),2.0-not_converted,14
2,1-(Other),converted,1
3,1-(Other),not_converted,40
4,1-Affiliates,2.0-Affiliates,3


In [101]:
# flows is a list of flows, where each flow is a tuple of (source, target, value)
flows = [(source, target, count) for source, target, count in zip(flows_df['source'], flows_df['target'], flows_df['id'])]
flows

[('1-(Other)', '2.0-Organic Search', 1),
 ('1-(Other)', '2.0-not_converted', 14),
 ('1-(Other)', 'converted', 1),
 ('1-(Other)', 'not_converted', 40),
 ('1-Affiliates', '2.0-Affiliates', 3),
 ('1-Affiliates', '2.0-not_converted', 1145),
 ('1-Affiliates', 'converted', 6),
 ('1-Affiliates', 'not_converted', 12411),
 ('1-Direct', '2.0-Direct', 404),
 ('1-Direct', '2.0-Display', 5),
 ('1-Direct', '2.0-Organic Search', 42),
 ('1-Direct', '2.0-Paid Search', 4),
 ('1-Direct', '2.0-Referral', 178),
 ('1-Direct', '2.0-Social', 2),
 ('1-Direct', '2.0-not_converted', 10639),
 ('1-Direct', 'converted', 1434),
 ('1-Direct', 'not_converted', 102886),
 ('1-Display', '2.0-Display', 25),
 ('1-Display', '2.0-Organic Search', 11),
 ('1-Display', '2.0-Paid Search', 2),
 ('1-Display', '2.0-Referral', 18),
 ('1-Display', '2.0-not_converted', 519),
 ('1-Display', 'converted', 76),
 ('1-Display', 'not_converted', 2450),
 ('1-Organic Search', '2.0-Direct', 12),
 ('1-Organic Search', '2.0-Display', 15),
 ('1-Or

In [102]:
!pip install sankeyflow

In [103]:
!pip install matplotlib

In [104]:
import matplotlib.pyplot as plt

In [105]:
from sankeyflow import Sankey

plt.figure(figsize=(4, 3), dpi=144)
s = Sankey(flows=flows, nodes=nodes)
s.draw()

KeyError: "Bad flow - couldn't find destination node: ('1-(Other)', '2.0-Organic Search', 1)"

<Figure size 576x432 with 0 Axes>

In [71]:
# create the nodes (each step counts) and flows (step to step counts)

import pandas as pd
import plotly.graph_objects as go


class CustomerJourneySankey:
    def __init__(self, journey_df):
        self.journey_df = journey_df
    
    def create_sankey_diagram(self, n_steps=5, title='Customer Journey Sankey Diagram'):
        # Get the maximum journey_order_number
        max_steps = self.journey_df['journey_order_number'].max()
        n_steps = min(n_steps, max_steps)
        
        # Add conversion_status column
        self.journey_df['conversion_status'] = self.journey_df['total_purchase'].apply(lambda x: 'Converted' if x > 0 else 'Not-Converted')
        
        # Create a list of DataFrames for each step
        step_dfs = []
        for step in range(1, n_steps + 1):
            step_df = self.journey_df[self.journey_df['journey_order_number'] == step]
            step_df = step_df.groupby(['state', 'next_state']).size().reset_index(name='count')
            step_dfs.append(step_df)
        
        # Add the final conversion_status step
        final_step_df = self.journey_df.groupby(['state', 'conversion_status']).size().reset_index(name='count')
        step_dfs.append(final_step_df)
        
        # Create a dictionary to map states to unique numeric labels
        state_labels = list(set(self.journey_df['state'].unique()) | set(self.journey_df['next_state'].unique()) | set(['Converted', 'Not-Converted']))
        state_labels = [str(label) for label in state_labels if pd.notnull(label)]
        state_dict = {label: i for i, label in enumerate(state_labels)}
        
        # Create the Sankey diagram
        fig = go.Figure(data=[go.Sankey(
            node = dict(
                pad = 20,
                thickness = 20,
                line = dict(color = "black", width = 0.5),
                label = state_labels,
                color = "blue"
            ),
            link = dict(
                source = [state_dict[state] for step_df in step_dfs for state in step_df.iloc[:, 0]],
                target = [state_dict[state] for step_df in step_dfs for state in step_df.iloc[:, 1]],
                value = [count for step_df in step_dfs for count in step_df['count']]
            )
        )])
        
        fig.update_layout(title_text=title, font_size=16)
        fig.show()

In [72]:
# Create an instance of CustomerJourneySankey
sankey = CustomerJourneySankey(journey_df)

# Create and display the Sankey diagram
sankey.create_sankey_diagram()

In [63]:
import plotly.graph_objects as go
import pandas as pd

class CustomerJourneySankey:
    def __init__(self, journey_df):
        self.journey_df = journey_df
    
    def create_sankey_diagram(self):
        # Create a DataFrame with the required columns for Sankey diagram
        sankey_df = self.journey_df.groupby(['state', 'next_state']).size().reset_index(name='count')
        
        # Create a dictionary to map states to unique numeric labels
        state_labels = list(set(sankey_df['state'].unique()) | set(sankey_df['next_state'].unique()))
        state_labels = [str(label) for label in state_labels if pd.notnull(label)]
        state_dict = {label: i for i, label in enumerate(state_labels)}
        
        # Create a list of colors for each state
        colors = ["blue" if state == "Paid Search" else "green" for state in state_labels]
        
        # Create the Sankey diagram
        fig = go.Figure(data=[go.Sankey(
            node = dict(
                pad = 15,
                thickness = 20,
                line = dict(color = "black", width = 0.5),
                label = state_labels,
                color = colors
            ),
            link = dict(
                source = [state_dict[state] for state in sankey_df['state']],
                target = [state_dict[state] for state in sankey_df['next_state']],
                value = sankey_df['count']
            )
        )])
        
        fig.update_layout(title_text="Customer Journey Sankey Diagram", font_size=10)
        fig.show()

In [64]:

# Create an instance of CustomerJourneySankey
sankey = CustomerJourneySankey(journey_df)

# Create and display the Sankey diagram
sankey.create_sankey_diagram()

In [ ]:
import plotly.graph_objects as go
import urllib, json

url = 'https://raw.githubusercontent.com/plotly/plotly.js/master/test/image/mocks/sankey_energy.json'
response = urllib.request.urlopen(url)
data = json.loads(response.read())
# show the first 5 example data
data['data'][:5]

# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity
data['data'][0]['node']['color'] = ['rgba(255,0,255, 0.8)' if color == "magenta" else color for color in data['data'][0]['node']['color']]
data['data'][0]['link']['color'] = [data['data'][0]['node']['color'][src].replace("0.8", str(opacity))
                                    for src in data['data'][0]['link']['source']]

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "TWh",
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  data['data'][0]['node']['label'],
      color =  data['data'][0]['node']['color']
    ),
    # Add links
    link = dict(
      source =  data['data'][0]['link']['source'],
      target =  data['data'][0]['link']['target'],
      value =  data['data'][0]['link']['value'],
      label =  data['data'][0]['link']['label'],
      color =  data['data'][0]['link']['color']
))])

fig.update_layout(title_text="Energy forecast for 2050<br>Source: Department of Energy & Climate Change, Tom Counsell via <a href='https://bost.ocks.org/mike/sankey/'>Mike Bostock</a>",
                  font_size=10)
fig.show()